In [ ]:
#Dependencies
!pip install transformers
!pip install bert-extractive-summarizer

!pip install spacy
!pip install neuralcoref



In [ ]:
import torch
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
from transformers import BertTokenizer
from transformers import BertForNextSentencePrediction


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokens=tokenizer.tokenize(case_doc_100)
print('Tokenized: ', tokens)

encoded=tokenizer.encode(
                        case_doc_100,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )

print(encoded)
print("Length encoded",len(encoded))

Tokenized:  ['state', 'of', 'uttar', 'pradesh', 'and', 'others', 'v', 'des', '##h', 'raj', 'supreme', 'court', 'of', 'india', '23', 'november', '2006', 'appeal', '(', 'civil', ')', '56', '##7', '##4', 'of', '2006', 'the', 'judgment', 'was', 'delivered', 'by', ':', 's', '.', 'b', '.', 'sin', '##ha', ',', 'j', '.', 'leave', 'granted', '.', 'it', 'is', 'also', 'doubtful', 'as', 'to', 'whether', 'the', 'imp', '##ug', '##ned', 'directions', 'could', 'have', 'been', 'issued', 'even', 'at', 'the', 'final', 'hearing', 'of', 'the', 'matter', 'which', 'would', 'amount', 'to', 'creation', 'of', 'super', '##num', '##era', '##ry', 'post', 'in', 'purported', 'compliance', 'of', 'the', 'regular', '##isation', 'rules', '.', 'an', 'appointment', 'which', 'was', 'made', 'throwing', 'all', 'constitutional', 'obligations', 'and', 'statutory', 'rules', 'to', 'winds', 'would', 'render', 'the', 'same', 'illegal', 'whereas', 'irregular', '##ity', 'pre', 'suppose', '##s', 'substantial', 'compliance', 'of', 'th

In [ ]:
#Ref— https://medium.com/runwayml/whats-my-line-next-sentence-prediction-in-runway-ad76cbf28c86
#Note this is just to explain the intuition behind going for BERT 
#The embeddings for doc/ query was actually collected using bert-extractive-summarizer

from transformers import BertTokenizer
from transformers import BertForNextSentencePrediction
import torch


def summarize_text(text):
  model = Summarizer()
  summarised = model(text, min_length=100,num_sentences=2)

  return summarised
def sequence_score(inputs):
	model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
	tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
	line1 = inputs['query']
	documents = inputs['docs']
	loss_scores = []
	for i,doc in enumerate(documents):
		combined = inputs['query'] + ' ' + doc
		input_tokens = tokenizer.encode(combined, add_special_tokens=True)
		input_ids = torch.tensor(input_tokens).unsqueeze(0)
		outputs = model(input_ids)
		sequence_loss = outputs[0][0][0]     # we take the first bit output
		sequence_loss = float(sequence_loss.cpu().detach().numpy())
		loss_scores.append(sequence_loss)
		print("Doc "+str(i)+"=>", sequence_loss)
	return loss_scores



query="fights between landlords and farmers"
doc11="legal disputes"
doc22="massive killings"
inputs={'query':query,'docs':docs}
sequence_score(inputs)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Doc 0=> 5.153006553649902
Doc 1=> 5.55983829498291


[5.153006553649902, 5.55983829498291]

In [ ]:
#Text Summarisation of Case Docs 

import pandas as pd
from google.colab import files
import io
from collections import defaultdict
from summarizer import Summarizer

model = Summarizer()


doc0="""
Masud Khan v State Of Uttar Pradesh
Supreme Court of India

26 September 1973
Writ Petition No. 117 of 1973
The Judgment was delivered by : A. Alagiriswami, J.
1.  Petitioner Masud Khan prays for his release on the ground that he, an Indian citizen has been illegally arrested and confined to, jail under Paragraph 5 of the Foreigners (Internment) Order, 1962. He had come to India from Pakistan on the basis of a Pakistani passport dated 137-1954and Indian visa dated 9-4-1956. In his application for visa he had stated that he had migrated to Pakistan in 1948 and was in Government service in Pakistan in P.W.D. as a Darogha and had given his permanent address as Hyderabad (Sind).
2.  If these statements were correct the petitioner would clearly be a Pakistani national. When this fact was brought out in the counter affidavit filled on behalf of the respondent, the petitioner filed a further affidavit stating that he was appointed as a Police Constable in Hasanganj Police Station, District Fatehpur, U.P. in February 1947 and continued as a Police Constable till the middle of 1950 when he was dismissed from service, and that he went to Pakistan in the year 1951.In the reply affidavit filed on behalf of the respondent it is stated that one Md. Masood Khan son of Zahoor Khan was enrolled as Police Constable on 16-9-1947 and he was discharged from service on 20-5-1949. It is fairly clear that this information culled from the English Order Book from 1-101947 to 27-12-1951 refers to the petitioner. While, therefore, it is established that the petitioner did not go to Pakistan in 1948, it cannot be said that it has been established that the petitioner went to Pakistan only in 1951.When he went to Pakistan is a matter peculiarly within his knowledge and the produced no evidence in support of that statement.
3.  Considering the frequent change of ground which the petitioner has resorted to, a mere statement from him cannot be accepted as true. Nor can we accept his contention that it is for the respondent to establish that lie did not go to Pakistan in 1951 but that he went on some other date. The petitioner has also alleged that he was married in U.P. on 25th December, 1949.Even assuming that this statement is correct; the petitioner cannot establish that he is a citizen of India unless lie succeeds in establishing that he was in India on 26-1-1950. If he bad been in India on 26-1-1950 but had gone to Pakistan in 1951 it would be for the Central Government to decide whether he is a Pakistani national or an Indian citizen even though he may have come to India on a Pakistani passport in 1956. That question does not arise here.
4.  We are not prepared to assume that the petitioner should be deemed to have been present in India on 26-1-1950, as was urged on behalf of the petitioner. There is no room for any such presumption. Under s-9 of the Foreigners Act whenever a question arises whether a person is or is not a foreigner the onus of proving that he is not a foreigner lies upon him. The burden is therefore, upon the petitioner to establish that be is a citizen of India in the manner claimed by him and therefore be is not a foreigner. This burden not having been discharged by the petitioner it should be held that he is a foreigner and his claim that he is an Indian citizen cannot be dealt with under the Foreigners (Internment) Order, 1962 must be rejected.
5.  It appears, however, that in 1960 he had been prosecuted before the Sub-Divisional Magistrate, Fatehpur under s. 14 of the Foreigner--, Act and was acquitted on the ground that he was not a foreigner. It was therefore contended that the question whether the petitioner is -a foreigner or not is a matter of issue estoppels. The decision that he was not a foreigner seems to have been based on the decision of the Allahabad High Court in Mohd. Hanif Khan v. State (AIR 1960 All. 434). 1959 Indlaw ALL 154It was held there that a Pakistani national who entered into India before the amendment to the Foreigners Act in 1957, when he could not be considered to be a foreigner, could not be so held because of that amendment. That decision was that of a learned Single Judge. On the point at issue he differed from an earlier decision of a learned Single Judge of the same Court in Ali Sher v. The State (AIR 1960 All. 431). 1959 Indlaw ALL 153But he decided that case before him on a different point and did not think it necessary to refer the case before him to a Bench for considering which of the two decisions was correct on the question regarding the nationality of a person who came to India on a Pakistani passport before 1957. There are thus two conflicting decisions of the same court on the same point and the Magistrate who decided the petitioner's case followed one of them.
6.  But that apart, this matter could bedecided on another point.. The question of issue-estoppels has been considered by this Court in Pritam Singh v. State, of Punjab (AIR1956 SC 415), 1955 Indlaw SC 111 Manipur Administration v. Thokchom, Bira Singh (1964 7 SCR 123) 1964 Indlaw SC 413 and Piara Singh S. State of Punjab. Issue-estoppels arises only if the earlier as well as the subsequent proceedings were criminal prosecutions. In the present case while the earlier one was a criminal prosecution the present is merely an action taken, under the Foreigners (Internment) Order for the purpose of deporting the petitioner out of India.
7.  It is nota criminal prosecution. The principle of issue estoppels is simply this : that where an issue of fact has been tried by a competent court on a former occasion and a finding has been reached in favored an accused, such a finding would constitute an estoppels or res judicator against the prosecution not as a bar to the trial and conviction of the accused for a different or distinct offence but as precluding the reception of evidence to disturb that finding of fact when the accused is tried subsequently, even for different offence which might be permitted by law. Pritam Singh's case 1955 Indlaw SC 111 (supra) was based on the decision of the Privy Council in Sambasivam v. Public, Prosecutor, Federation of Malaya (1950 A.C. 458). In that case Lord McDermott speaking for the Board said:
"The effect of a verdict of acquittal pronounced by a competent court on a lawful charge and after a lawful trial is not completely stated by saying that the person acquitted cannot be tried again for the same offence. To that it must be added that the verdict is binding and conclusive in all subsequent proceedings between the parties to the adjudication."
It should bekept clearly in mind that the proceeding referred to herein is a criminal prosecution. The plea of issue-estoppels not the same as the, plea of double jeopardy or aura foist acquit. In The King v. Wilkes (77 C.L.R.511)
Divon, J.
8.  Referring to the question of issue estoppel said. view that there is an issue estoppels, if it appears by record of itself or as explained by proper evidence, that the same point was determined in favour of a prisoner in a previous criminal trial which is brought in issue on a second criminal trial of the same prisoners There must be prior proceeding determined against the Crown necessarily involving an issue which again arises in a subsequent proceeding by the Crown against the same prisoner. The allegation of the Crown in the subsequent proceeding must itself be inconsistent with the acquittal of the prisoner in the previous proceeding. But if such a condition of affairs arises I see no reason why the ordinary rules of issue-estoppels should not apply Issue-estoppels concerned with the judicial establishment of a proposition of law or fact between parties. It depends upon well-known doctrines which control the reiteration of issues which are settled by prior litigation."
"The emphasis here again would be seen to be on the determination of, criminal liability. In Marz v. The Queen (96 C.L.R. 62) the High Court of Australia said "The Crown is as much precluded by an estoppels by judgment in criminal proceedings as is a subject in civil proceedings The laws which gives effect to issue estoppels is not concerned with the correctness or incorrectness of the finding which amounts to an estoppels, still less with the process of reasoning by which the finding was reached. in fact It is enough that an issue or issues have been distinctly raised or found. Once that is done, then, so long as the finding stands, if there be any subsequent litigation between the same parties, no allegations legally inconsistent with the finding, may be made by one of them against the other."
Here again it is to be remembered that the principle applies to two criminal proceedings and the proceeding with which we are now concerned is not a criminal proceeding. We therefore hold that there is no substance in this contention.

"""


query="""The appellant on February 9, 1961 was appointed as an Officer in Grade III in the respondent Bank ( for short 'the Bank'). He was promoted on April 1, 1968 to the Grade officer in the Foreign Exchange Department in the Head Office of the Bank. Sometime in 1964, MCH Society ( for short 'the Society') was formed of which the appellant was one of the chief promoters and thereafter its Secretary. The object of the Society was to construct residential premises for the employees of the Bank and its other members. It appears that the complaint was received in respect of the affairs of the Society relating to misappropriation of the funds of the Society and consequently, in exercise of the powers under Section S of Act A1, the Registrar on April 23, 1969 instituted an inquiry thereof. P1 was appointed the Registrar's nominee who on October 4, 1969; submitted the report holding the appellant and two other office bearers of the Society negligent in dealing with the funds of the Society causing a loss to the tune of Rs. 3,59,000/-. The Registrar on October 21, 1969, passed an order appointing an officer under Section S of A1 to assess the loss caused to the Society. However, the Government by its order dated November 29, 1969 annulled the Registrar's order dated April 23, 1969 and October 21, 1969 and directed a fresh inquiry into the affairs of the Society. On December 17, 1969, the Bank issued show cause notice to the appellant to explain within fifteen days his alleged negligent conduct in dealing with the affairs of the Society as revealed in the report dated 4th October, 1969. In the meantime, P2 came to be appointed by the Registrar vide his order dated 26th July, 1969, to make inquiries under Section S of A1. Petitioner by his reply dated 18/22th January, 1970 submitted his explanation and also challenged the legality of the inquiry and the findings recorded therein. On 5th March, 1970, P3, treasurer of the Society and an employee of the Bank criminal complaints in the Court of Addl. Chief Presidency Magistrate  alleging that the appellant and two other office bearers of the society had dishonestly misappropriated a sum of Rs. 51,000/ and Rs. 80,000/- respectively which was entrusted to the appellant in his capacity as Promoter and Secretary of the Society and thereby committed criminal breach of trust. The Magistrate framed the charges against the appellant. The Bank having regard to the serious misconduct of the appellant involving moral turpitude vide its order dated 3rd November, 1970 suspended the appellant pending trial. The appellant protested this action of the Bank complaining that he was not given an opportunity of hearing before passing the order of suspension. In the meantime, P2, the authorized officer appointed by the Registrar vide his order dated 9th October, 1971 held the appellant liable to pay Rs. 2,36,000/- to the Society in addition to the amount of Rs. 2,03,000/- for which he (the appellant) and two other office bearers of the Society were held jointly liable. The Bank in view of this finding, vide its order dated 29th November, 1971 terminated the services of the appellant with effect from 1st December, 1971 along with notice pay. The appellant protested against the Action of the Bank and on 3rd December, 1971 filed detailed representation against the order of termination. The Bank replied to the appellant's representation and justified its action. The appellant on 28th December, 1971 submitted his reply to the Bank stating, inter alia, that the termination of his services was not simplicitor and was in violation of the principles of natural justice; that no opportunity of hearing was given to him; that the termination order attached stigma. The appellant aggrieved by the findings and order made by P2 appealed before Tribunal. In the meantime, the criminal proceedings ended in conviction vide order dated 27th March, 1972 passed by the Addl. Chief Metropolitan Magistrate. The appellant challenged the order of conviction and sentence in the High Court  and during the pendency of the said appeal, the Tribunal vide its order dated April 12, 1973 dismissed the appellant's appeal but reduced the liability by Rs. 72,000/-. On November 12, 1973, the High Court allowed the criminal appeal and acquitted the appellant. The High Court, however, in its order observed that since the services of the appellant were terminated in view of the criminal proceedings and since the appellant has been acquitted, representation, if a any, by the appellant to the Bank for reinstatement may be considered sympathetically. Taking clue from the observations made by the High Court, the appellant filed three representations, the last being dated 3rd May, 1975 requesting the Bank to revoke the order of termination and be reinstated. The Bank vide its communication dated May 21, 1975 refused to reinstate the appellant. The appellant, therefore, on July 23, 1975 filed the writ petition in the High Court for quashing the orders dated 29th November 1971, 27th December, 1971 and 21st May, 1975 passed by the Bank. The learned Single Judge of the High Court by his judgment and order dated December 6/7, 1979 granted desired relief to the appellant. The Bank aggrieved by the judgment and order passed by the learned Single Judge preferred an appeal under Clause 15 of the Letters Patent. The appeal was heard by the Division Bench. The Division Bench of the High Court did not agree with the judgment passed by the learned Single Judge and consequently by its judgment and order dated April 16 1985 allowed the appeal and dismissed the writ petition the ground of laches and also on merits. It is this judgment and order of the High Court which is impugned in this appeal."""

case_doc0_summ=model(query,num_sentences=3)
print("Summary of doc—",case_doc0_summ)
print()
print("Length of original doc—", len(doc0))
print("Length of original doc—", len(case_doc0_summ))
bert_embedding=model.run_embeddings(doc0, num_sentences=20,aggregate='mean')
print("The embedding is—",bert_embedding)
print(bert_embedding)


Summary of doc— The appellant on February 9, 1961 was appointed as an Officer in Grade III in the respondent Bank ( for short 'the Bank'). It appears that the complaint was received in respect of the affairs of the Society relating to misappropriation of the funds of the Society and consequently, in exercise of the powers under Section S of Act A1, the Registrar on April 23, 1969 instituted an inquiry thereof. On November 12, 1973, the High Court allowed the criminal appeal and acquitted the appellant. Taking clue from the observations made by the High Court, the appellant filed three representations, the last being dated 3rd May, 1975 requesting the Bank to revoke the order of termination and be reinstated.

Length of original doc— 8870
Length of original doc— 701
The embedding is— [-1.1194632  -0.47189993 -0.11958978 ...  0.10920823  0.31443277
 -0.3085639 ]
[-1.1194632  -0.47189993 -0.11958978 ...  0.10920823  0.31443277
 -0.3085639 ]


In [ ]:
import io
import pandas as pd
from google.colab import files

#Case docs
"""
uploaded = files.upload()
df= pd.read_csv(io.BytesIO(uploaded['case_docs.csv']))
print(df)
"""
uploaded = files.upload()
df= pd.read_csv(io.BytesIO(uploaded['case_docs_200.csv']))
print(df.columns)

In [ ]:
#Query doc
uploaded2 = files.upload()
qdf= pd.read_csv(io.BytesIO(uploaded2['query.csv']))
print(qdf)

In [ ]:
import nltk
import string
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 

nltk.download('punkt')  
nltk.download('stopwords')
nltk.download('wordnet')


def remove_punctuation(text):
    #also whitespace
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator)

def remove_stop_words_and_lemmatize(sentence):
  sentence=remove_punctuation(sentence)
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(sentence) 
  word_tokens=[word.lower() for word in word_tokens]
  filtered_sentence = [w for w in word_tokens if not w in stop_words]
  lemmatizer = WordNetLemmatizer()
  lemmatized_sentence=[lemmatizer.lemmatize(w) for w in filtered_sentence]  
  return lemmatized_sentence
  

def give_me_synonyms(word):
  synonyms=set()
  for synset in wordnet.synsets(word):
    for lemma in synset.lemmas():
      syn=lemma.name()
      synonyms.add(syn)
  return list(synonyms)

def expand_query(query,limit=500):
  query_processed=remove_stop_words_and_lemmatize(query)
  query_new=[]
  count=0
  for word_current in query_processed:
    query_new.append(word_current) #make sure the old word is there
    query_synonyms=give_me_synonyms(word_current)
    for word_new in query_synonyms:
      query_new.append(word_new)
      if(count>limit):
        return query_new
  
  return query_new

In [ ]:
#Store text summarized case docs, lemmatized case doc and embeddings !
import numpy as np
new_document=[]
embeddings=[]
for i, row in df.iterrows():
  print("processing doc",i+1)
  doc=row['original']
  lem_doc=row['lemmatized']
  #lem_doc=remove_stop_words_and_lemmatize(doc)
  #lem_doc=" ".join(lem_doc)
  #doc_type1=model(doc,min_length=50)
  #doc_type2=model(doc,num_sentences=1)
  embedding=model.run_embeddings(doc,aggregate='mean')
  embedding=np.reshape(embedding,(1,1024))
  embeddings.append(embedding)
  #new_document.append({"original":doc,"lemmatized":lem_doc,"summarized_256":doc_256}) #use a wrapper around the numpy array imp!

#download lemmatized and summarized text
"""
result_display=pd.DataFrame(new_document)
print(result_display)
result_display.to_csv('case_docs_200.csv') 
files.download('case_docs_200.csv')
"""

#download embeddings
embeddings=np.array(embeddings)
print(embeddings.shape)
np.save('embeddings.npy', embeddings)
files.download('embeddings.npy')

In [ ]:
#Get Query Embeddings

queries_list=[]
query_embeddings=[]
for i,row in qdf.iterrows():
  query=row["Query"]
  lem_query=" ".join(remove_stop_words_and_lemmatize(query))
  embedding=model.run_embeddings(query,aggregate='median')
  embedding=np.reshape(embedding,(1,1024))
  query_embeddings.append(embedding)
  queries_list.append({"index":row["Index"],"query":row['Query'],"lemmatized":lem_query,"relevant":rel_df.loc[i,"relevant"]})


query_display=pd.DataFrame(queries_list)
#print(query_display["embedding"])
query_display.to_csv('embedded_queries.csv') 
files.download('embedded_queries.csv')

#download embeddings
query_embeddings=np.array(query_embeddings)
print(query_embeddings.shape)
np.save('query_embeddings.npy', query_embeddings)
files.download('query_embeddings.npy')
